## Sales Prediction Agent using Langchain

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sqlalchemy import create_engine
import os,sys
from dotenv import load_dotenv

In [ ]:
try:
    load_dotenv()
    db_username = os.getenv("db_username")
    db_password = os.getenv("db_password")
    db_host = os.getenv("db_hostname")
    db_name = os.getenv("db_database")
    db_server = os.getenv("db_server")
    # print(db_username, db_password, db_name, db_server)

    connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
    engine = create_engine(connection_string)
except Exception as e:
    print("Error connecting to SQL Server:", e)
    sys.exit(1)

sa clouderp123! essentials 103.172.151.143


In [3]:
import pandas as pd
query = os.getenv("SQL_QUERY")

In [ ]:
query

In [4]:
df = pd.read_sql(query, engine)
engine.dispose()

In [5]:
df.columns

Index(['repcontactname', 'docuserfield1', 'docuserfield2', 'docuserfield3',
       'prodnetweight', 'prodgrossweight', 'docid', 'docstatusname',
       'docpaymentterms', 'docdeliveryterms', 'ewbnum', 'billnum', 'docrefnum',
       'docrefdate', 'billdate', 'contactname', 'docamount', 'exchangerate',
       'docamountinr', 'docgrossamount', 'docotherchargeamount',
       'docotherchargeamount_subtcs', 'productno', 'hsncode', 'proddisc',
       'prodvaluebeforedisc', 'proddocvalue', 'proddocvalueinr',
       'prodassessablevalue', 'prodassessablevalueinr', 'currencycode',
       'docamountcalculated', 'prodcontactqty', 'proddocqty',
       'prodcontactrate', 'uomname', 'stkuom', 'prodotherchargeamount',
       'igstper', 'igst', 'otherchargeamount', 'cgstper', 'cgst', 'sgstper',
       'sgst', 'gstn', 'tcsper', 'tcs', 'vesselnum', 'transportername',
       'lrnum', 'docotherchargeamountinr', 'consignee', 'consigneecountry',
       'finaldestinationport', 'prodname', 'doctypeindex', 'doc

In [ ]:
df

In [6]:
df.fillna(method='ffill', inplace=True)

C:\Users\eDominer\AppData\Local\Temp\ipykernel_17736\3970806690.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [7]:
df

,repcontactname,docuserfield1,docuserfield2,docuserfield3,prodnetweight,prodgrossweight,docid,docstatusname,docpaymentterms,docdeliveryterms,...,vesselnum,transportername,lrnum,docotherchargeamountinr,consignee,consigneecountry,finaldestinationport,prodname,doctypeindex,docnotes
0,None,NYK-5401328-2719192-1-1,28449169011705,NaN,NaN,NaN,6375287254735139681,Dispatched,None,None,...,None,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None
1,None,NYK-5401779-0181250-1-1,28449170020851,NaN,NaN,NaN,6375287255533577111,Dispatched,None,None,...,None,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None
2,None,NYK-5395241-1902625-1-1,28449170019786,NaN,NaN,NaN,6375287256177327461,Dispatched,None,None,...,None,None,None,0.0,None,None,None,Shade of Green Wooden Tray,192,None
3,None,NYK-5395979-5429875-4-1,28449170016813,NaN,NaN,NaN,6375287256877327371,Dispatched,None,None,...,None,None,None,0.0,None,None,None,Knurl Terracotta Curd Setter(Small),192,None
4,None,NYK-5399415-9567568-3-1,28449169010386,NaN,NaN,NaN,6375287257619515131,Dispatched,None,None,...,None,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227218,Ellementry HYD,124115482,35907710001411,1.0,5.99,8.9,6384370149715285401,Dispatched,Advance,FOB,...,DL1LAH7488,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Terracotta Water Bottle With Sphere Stopper,192,Website offer
227219,Ellementry Lucknow,Ellementry Store,35907710001411,1.0,5.99,8.9,6384370926108058071,Dispatched,Advance,FOB,...,DL1LAH7488,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Golden foliage wooden napkin holder set of 6,192,Website offer
227220,Ellementry CG,Ellementry Store,35907710001411,1.0,5.99,8.9,6384371588792208021,Dispatched,Advance,FOB,...,DL1LAH7488,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Clear Glass Bowl With Wooden Stand,192,Website offer
227221,Ellementry CG,Ellementry Store,35907710001411,1.0,5.99,8.9,6384371588792208021,Dispatched,Advance,FOB,...,DL1LAH7488,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Drink More Glass Water Bottle With Wooden Stopper,192,Website offer


#### Prepare Data for Prophet model

In [8]:
df['billdate'] = pd.to_datetime(df['billdate'], format='%d/%m/%Y')
df['unique_order'] = df['billdate'].astype(str) + '_' + df['productno'].astype(str)

In [9]:
df

,repcontactname,docuserfield1,docuserfield2,docuserfield3,prodnetweight,prodgrossweight,docid,docstatusname,docpaymentterms,docdeliveryterms,...,transportername,lrnum,docotherchargeamountinr,consignee,consigneecountry,finaldestinationport,prodname,doctypeindex,docnotes,unique_order
0,None,NYK-5401328-2719192-1-1,28449169011705,NaN,NaN,NaN,6375287254735139681,Dispatched,None,None,...,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None,2022-04-01_TCKEA0923
1,None,NYK-5401779-0181250-1-1,28449170020851,NaN,NaN,NaN,6375287255533577111,Dispatched,None,None,...,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None,2022-04-01_TCKEA0923
2,None,NYK-5395241-1902625-1-1,28449170019786,NaN,NaN,NaN,6375287256177327461,Dispatched,None,None,...,None,None,0.0,None,None,None,Shade of Green Wooden Tray,192,None,2022-04-01_WDTEA2131
3,None,NYK-5395979-5429875-4-1,28449170016813,NaN,NaN,NaN,6375287256877327371,Dispatched,None,None,...,None,None,0.0,None,None,None,Knurl Terracotta Curd Setter(Small),192,None,2022-04-01_TCKEA2335
4,None,NYK-5399415-9567568-3-1,28449169010386,NaN,NaN,NaN,6375287257619515131,Dispatched,None,None,...,None,None,0.0,None,None,None,Terracotta Curd Setter With Wooden Lid- Small,192,None,2022-04-01_TCKEA0923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227218,Ellementry HYD,124115482,35907710001411,1.0,5.99,8.9,6384370149715285401,Dispatched,Advance,FOB,...,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Terracotta Water Bottle With Sphere Stopper,192,Website offer,2025-02-16_TCTEA0930
227219,Ellementry Lucknow,Ellementry Store,35907710001411,1.0,5.99,8.9,6384370926108058071,Dispatched,Advance,FOB,...,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Golden foliage wooden napkin holder set of 6,192,Website offer,2025-02-16_MPTEA2748
227220,Ellementry CG,Ellementry Store,35907710001411,1.0,5.99,8.9,6384371588792208021,Dispatched,Advance,FOB,...,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Clear Glass Bowl With Wooden Stand,192,Website offer,2025-02-16_GSTEA1832
227221,Ellementry CG,Ellementry Store,35907710001411,1.0,5.99,8.9,6384371588792208021,Dispatched,Advance,FOB,...,Delhivery ELLEMENTRYBULKY SURFACE,,0.0,Shifteco FZ-LLC,UNITED ARAB EMIRATES,Atlanta,Drink More Glass Water Bottle With Wooden Stopper,192,Website offer,2025-02-16_GSTEA0996


In [10]:
wholesale_data = df.groupby('billdate')['docamountinr'].sum().reset_index()
wholesale_data.columns = ['ds', 'y']

In [11]:
wholesale_data = wholesale_data.dropna(subset=['ds', 'y'])

In [12]:
wholesale_data

,ds,y
0,2022-04-01,193414.02
1,2022-04-02,210022.25
2,2022-04-03,267619.08
3,2022-04-04,256253.80
4,2022-04-05,391464.84
...,...,...
1045,2025-02-12,661189.88
1046,2025-02-13,442969.71
1047,2025-02-14,326204.11
1048,2025-02-15,328288.82


-----New Prophet Model Checking---

In [ ]:
import os
import pandas as pd
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from prophet import Prophet
from pydantic import BaseModel

class ProphetTool(Tool):
    def __init__(self, name: str, func, description: str, model: Prophet):
        super().__init__(name=name, func=func, description=description)
        self.model = model

    def run(self, periods):
        # Create a DataFrame for future dates
        future = self.model.make_future_dataframe(periods=periods)
        forecast = self.model.predict(future)
        return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Initialize your Prophet model
model = Prophet()
# Fit your model with the training data
model.fit(wholesale_data)

# Create a custom tool for the Prophet model
prophet_tool = ProphetTool(
    name="Prophet Forecast Tool",
    func=lambda periods: prophet_tool.run(periods),  # Use a lambda to call the run method
    description="A tool for forecasting using the Prophet model.",
    model=model
)

# Update the toolkit
toolkit = {
    'forecast': prophet_tool,
    # Add other tools as needed, e.g., 'database': engine
}

tools = list(toolkit.values())

# Initialize the agent
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
agent = initialize_agent(llm=llm, tools=tools)

-----New Prophet Model Checking End---

In [13]:
model = Prophet()
model.fit(wholesale_data)

15:39:41 - cmdstanpy - INFO - Chain [1] start processing
15:39:41 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
# Make future predictions
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

In [15]:
forecast[['ds', 'yhat', 'yhat_lower',  'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2022-04-01,427553.457539,-284164.243169,1.169836e+06
1,2022-04-02,466492.820731,-213897.393513,1.162799e+06
2,2022-04-03,86394.797096,-675078.926297,7.322097e+05
3,2022-04-04,526728.554805,-183654.250657,1.241628e+06
4,2022-04-05,487355.228507,-252423.288551,1.203571e+06
...,...,...,...,...
1075,2025-03-14,716905.144591,-5016.597110,1.396822e+06
1076,2025-03-15,746073.057266,55983.800278,1.448564e+06
1077,2025-03-16,355353.698887,-315530.723472,1.091497e+06
1078,2025-03-17,784599.039751,73903.336738,1.454421e+06


### Building An Agent using Langchain

In [16]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"))

##### Calculate mean and standard deviation

##### Set threshold as a certain number of standard deviations below the mean

In [18]:
# Calculate the 10th percentile as the threshold
threshold = wholesale_data['y'].quantile(0.10)
print(f'Threshold: {threshold}')


Threshold: 161728.27000000002


In [19]:
def generate_recommendations(forecast):
    recommendations = []
    for index, row in forecast.iterrows():
        if row['yhat'] < threshold:
            recommendations.append(f"Move product {row['product_id']} to a different channel.")
    return recommendations


In [20]:
connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

##### Define the agent's toolkit

In [21]:
toolkit = {
    'forecast': model,
    'database': engine
}

In [22]:
tools = list(toolkit.values())

##### Create the agent

In [23]:
agent = initialize_agent(llm=llm, tools=tools)

C:\Users\eDominer\AppData\Local\Temp\ipykernel_17736\2670483501.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(llm=llm, tools=tools)


AttributeError: 'Prophet' object has no attribute 'is_single_input'